 python version：3.8   
 modules：pandas openpyxl

读取原始交易表单

In [7]:
import pandas as pd
filename="BINANCE_SWAP_BTC-USDT订单状态列表-data-2022-08-26 10_22_34.csv"
head_row = pd.read_csv(filename, nrows=0)
csv_result = pd.read_csv(filename, usecols=head_row)

处理

In [2]:
'''订单的状态属性
Time                时间信息
purpose             ['close' 'sync' 'open'] 
status              ['PENDING' 'OPENED' 'PARTIAL_FILLED' 'CANCELED' 'FULL_FILLED']
order_id            订单编号
client_id           一次交易完整周期的编号
source_type         信息发生来源
currency            币类型
'''

'''订单发生时的市场属性
price               币的价格
'''

'''订单中体现决策的属性
size                订单下单大小
filled_size         订单实际成交量
usd_size            一笔订单的美金价值
'''

choose_col=['Time','purpose','status','client_id','source_type','currency','price','filled_size','size','usd_size']
df=csv_result[choose_col]
start=csv_result[choose_col].drop_duplicates(subset=['client_id'],keep='first').reset_index(drop=True)
tot=df[(df.status=='CANCELED')|(df.status=='FULL_FILLED')].reset_index(drop=True)
tot['T']=(pd.to_datetime(tot.Time)-pd.to_datetime(start.iloc[0:len(tot)].Time)).dt.seconds
tot['start time']=start.iloc[0:len(tot)].Time

## 实际美元变动
tot['filled_usd_size']= tot['usd_size'] * tot['filled_size']/tot['size']
# 通过 loc 将FTT交易对美元资产影响排除
tot.loc[tot['currency'] == 'FTT', 'filled_usd_size'] = 0

保存处理结果

In [19]:
import os
if not os.path.exists('../cache/'):
    os.makedirs('../cache/')
tot.to_excel('../cache/交易结果表单2.xlsx',index=False)